In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv

In [17]:
N = 23 # Numero de periodos
mYah = 3 # Numero de acciones extraidas de Yahoo
mFin = 4 # Numero de acciones extraidas de Finance.com
m = mYah + mFin # Numero de acciones

precios = np.zeros((m, N+1))
retornos = np.zeros((m, N))

Extrayendo columnas de los archivos de Yahoo Finance:

In [4]:
archivosYahoo = ['csvs/AVH.csv', 'csvs/CIB.csv', 'csvs/EC.csv']

In [18]:
for k in range(mYah):
    archivo = archivosYahoo[k]
    lector = csv.reader(open(archivo))
    lector.__next__() #Ignorar primer renglon
    
    for i in range(N+1):
        precios[k, i] = lector.__next__()[5] #Extraer la 5ta columna: precio de cierre ajustado
        if (i > 0):
            retornos[k, i-1] = (precios[k, i] - precios[k, i-1])/precios[k, i-1]

Extrayendo columnas de los archivos de Investing.com

In [19]:
 archivosFin = ['csvs/ARG-2.csv', 'csvs/CFV-2.csv', 'csvs/FTSE-2.csv', 'csvs/IMI-2.csv']

In [20]:
for k in range(mYah, mYah + mFin):
    archivo = archivosFin[k - mYah]
    lector = csv.reader(open(archivo))
    lector.__next__() #Ignorar primer renglon
    
    for i in range(N, -1, -1):
        precios[k, i] = lector.__next__()[1] #Extraer la 1era columna: precio de cierre
        if (i < N):
            retornos[k, i] = (precios[k, i+1] - precios[k, i])/precios[k, i]

In [24]:
np.shape(retornos)

(7, 23)

TODO: Explicar, a la luz de la lectura 1, la naturaleza de esos activos.

# 3

Vector de Rendimientos promedio

In [45]:
retProm = np.mean(retornos, 1, keepdims = True) # Hallar el promedio de la matriz de retornos a lo largo del eje temporal
np.shape(retProm) # En efecto el vector retProm tiene tamaño 7x1, donde 7 es el número de activos
print("Retornos promedio:\n", retProm)

Retornos promedio:
 [[-0.02652642]
 [ 0.01816381]
 [ 0.03172053]
 [-0.00337518]
 [ 0.00148386]
 [ 0.01086576]
 [ 0.0050164 ]]


Matriz de Covarianzas

In [55]:
S = np.zeros((m, m)) # Inicializacion en 0's

for k in range(m):
    for l in range(m):
        for i in range(N):
            S[k, l] += (retornos[k, i] - retProm[k])*(retornos[l, i] - retProm[l]) # Para la combinacion de activos k y l
            
print("Matriz de covarianzas:\n", S)

Matriz de covarianzas:
 [[ 0.3059979   0.05765147  0.1618752  -0.02700894  0.0585867   0.02823461
   0.05425649]
 [ 0.05765147  0.13295584  0.12667706  0.05594031  0.08104902  0.06826378
   0.07474869]
 [ 0.1618752   0.12667706  0.39699286 -0.01626918  0.08986482  0.10578535
   0.08819466]
 [-0.02700894  0.05594031 -0.01626918  0.06872598  0.02678615  0.02218467
   0.02893063]
 [ 0.0585867   0.08104902  0.08986482  0.02678615  0.19584113  0.06972935
   0.07060562]
 [ 0.02823461  0.06826378  0.10578535  0.02218467  0.06972935  0.053816
   0.04258707]
 [ 0.05425649  0.07474869  0.08819466  0.02893063  0.07060562  0.04258707
   0.08397729]]
